## Imports

In [ ]:
import json
import pandas as pd
import altair as alt
from itables import init_notebook_mode
import ipywidgets as widgets

init_notebook_mode(all_interactive=True)

## Load data

In [ ]:
# cardano_node_0_df, cardano_node_0_df and all_df
def df_from_cardano_node_logs(fp):
    lines = open(fp, "r").readlines()
    at_lines = [ json.loads(line) for line in lines if line.startswith('{"at') ]
    return pd.DataFrame.from_records(at_lines)

def events_chart(df, eventPrefix):
    return alt.Chart(df[df.ns.str.startswith(eventPrefix)].reset_index()).mark_point().encode(
        x='at:T',
        y='source',
        color='ns',
        tooltip='ns'
    ).interactive()

cardano_node_0_df = df_from_cardano_node_logs("data/cardano-node-0.log")
cardano_node_1_df = df_from_cardano_node_logs("data/cardano-node-1.log")

all_df = pd.concat([cardano_node_0_df.assign(source="cardano-node-0"), cardano_node_1_df.assign(source="cardano-node-1")])

all_df

## Quick refresher

In [ ]:
# ChainSync events

events_chart(all_df, 'ChainSync')

In [ ]:
# SELECT * FROM all_df WHERE sev = 'Error'
display(events_chart(all_df[all_df.sev == "Error"], ""))
all_df[all_df.sev == "Error"]

In [ ]:
# SELECT DISTINCT_COUNT(ns), ARG_MAX(ns), ARG_MIN(ns), ARG_MAX(sev), ARG_MIN(sev) FROM all_df GROUP BY source 
all_df.groupby('source').agg({
    'ns': ['nunique', 'max', 'min'],
    'sev': ['max', 'min']
})

## Latency

In [ ]:
latency_logs_df = all_df[all_df.ns.isin(["BlockFetch.Client.CompletedBlockFetch", "ChainSync.Client.DownloadedHeader"])]
latency_logs_df = latency_logs_df.assign(
    block=latency_logs_df.data.apply(lambda r: r['block']),
)

In [ ]:
latency_logs_df.groupby(['block', 'source', 'ns']).agg({
    'at': ['count', 'min', 'max']
})